# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 106387657, 'calculationPrice': 'tops', 'change': 0.34, 'changePercent': 0.0019, 'close': 0, 'closeSource': 'ffoilaic', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 176.2, 'iexAskSize': 411, 'iexBidPrice': 179.48, 'iexBidSize': 405, 'iexClose': 180.14, 'iexCloseTime': 1723519768658, 'iexLastUpdated': 1658551865682, 'iexMarketPercent': 0.009659203295264937, 'iexOpen': 181.53, 'iexOpenTime': 1687414059669, 'iexRealtimePrice': 177.55, 'iexRealtimeSize': 101, 'iexVolume': 531286, 'lastTradeTime': 1717001605605, 'latestPrice': 173.16, 'latestSource': 'IEX real time price', 'latestTime': '1:59:09 PM', 'latestUpdate': 1710063754597, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2834570603702, 'oddLotDelayed

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

15.49

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
                            pd.Series([symbol, 
                                       data[symbol]['quote']['latestPrice'],
                                       data[symbol]['quote']['peRatio'],
                                       'N/A'
                                       ], 
                                      index = my_columns), 
                            ignore_index = True)
        
    
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,148.180,19.4,N/A
1,AAL,19.790,-4.0,N/A
2,AAP,249.161,24.68,N/A
3,AAPL,175.970,15.78,N/A
4,ABBV,137.570,32.62,N/A
...,...,...,...,...
500,YUM,140.530,27,N/A
501,ZBH,133.320,33.57,N/A
502,ZBRA,542.300,34.75,N/A
503,ZION,70.610,6.93,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_df.sort_values('Price-to-Earnings Ratio', inplace=True)
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index(inplace = True)
final_df.drop('index', axis =1, inplace = True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,670.240,3.11,N/A
1,HPE,17.330,3.46,N/A
2,COO,425.010,3.51,N/A
3,EBAY,69.820,3.67,N/A
4,HPQ,40.400,3.73,N/A
5,NRG,41.400,4.15,N/A
6,DHI,100.010,4.28,N/A
7,HOLX,71.950,4.98,N/A
8,BEN,35.381,5.08,N/A
9,MPC,73.450,5.1,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:25000



You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_df.index)
for row in final_df.index:
    final_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[row, 'Price'])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,670.240,3.11,0
1,HPE,17.330,3.46,28
2,COO,425.010,3.51,1
3,EBAY,69.820,3.67,7
4,HPQ,40.400,3.73,12
5,NRG,41.400,4.15,12
6,DHI,100.010,4.28,4
7,HOLX,71.950,4.98,6
8,BEN,35.381,5.08,14
9,MPC,73.450,5.1,6


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#data.status_code
#print(data['AAPL']['advanced-stats'])
pe_ratio = data[symbol]['quote']['peRatio']

pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

ps_ratio = data['AAPL']['advanced-stats']['priceToSales']
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker', 
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV / EBITDA',
    'EV / EBITDA Percentile',
    'EV / GP',
    'EV / GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_df = rv_df.append(
            pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'],
        index=rv_columns),
        ignore_index=True
            )
    

In [13]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,RV Score
0,A,149.140,N/A,19.28,N/A,4.37,N/A,3.73,N/A,14.521552,N/A,6.981359,N/A,N/A
1,AAL,19.525,N/A,-4.03,N/A,-1.66,N/A,0.5121,N/A,-7.630454,N/A,1.476193,N/A,N/A
2,AAP,246.330,N/A,24.92,N/A,4.76,N/A,1.41,N/A,14.000232,N/A,3.122170,N/A,N/A
3,AAPL,178.740,N/A,15.83,N/A,22.38,N/A,3.92,N/A,12.533348,N/A,9.767116,N/A,N/A
4,ABBV,135.670,N/A,33.42,N/A,18.03,N/A,4.42,N/A,11.718143,N/A,8.119916,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.350,N/A,27.12,N/A,-5.14,N/A,6.53,N/A,23.160349,N/A,10.702701,N/A,N/A
501,ZBH,131.930,N/A,33.23,N/A,2.2,N/A,3.5,N/A,19.277595,N/A,5.968519,N/A,N/A
502,ZBRA,553.240,N/A,34.79,N/A,10.87,N/A,5.65,N/A,28.032851,N/A,11.710455,N/A,N/A
503,ZION,70.860,N/A,6.78,N/A,1.49,N/A,3.1,N/A,5.368032,N/A,2.883805,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:

rv_df.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV / EBITDA',
       'EV / EBITDA Percentile', 'EV / GP', 'EV / GP Percentile', 'RV Score'],
      dtype='object')

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV / EBITDA', 'EV / GP']:
    rv_df[column].fillna(rv_df[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV / EBITDA': 'EV / EBITDA Percentile',
    'EV / GP': 'EV / GP Percentile'}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = score(rv_df[metric], rv_df.loc[row, metric])
        
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,RV Score
0,A,149.140,N/A,19.28,37.425743,4.37,56.534653,3.7300,54.455446,14.521552,48.118812,6.981359,48.712871,N/A
1,AAL,19.525,N/A,-4.03,5.148515,-1.66,5.544554,0.5121,5.148515,-7.630454,2.376238,1.476193,6.336634,N/A
2,AAP,246.330,N/A,24.92,52.079208,4.76,59.80198,1.4100,19.60396,14.000232,46.336634,3.122170,16.435644,N/A
3,AAPL,178.740,N/A,15.83,28.910891,22.38,93.861386,3.9200,57.425743,12.533348,39.80198,9.767116,68.118812,N/A
4,ABBV,135.670,N/A,33.42,65.940594,18.03,92.079208,4.4200,62.970297,11.718143,36.039604,8.119916,57.227723,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.350,N/A,27.12,55.643564,-5.14,5.148515,6.5300,78.613861,23.160349,73.465347,10.702701,72.277228,N/A
501,ZBH,131.930,N/A,33.23,65.544554,2.20,31.089109,3.5000,51.485149,19.277595,65.940594,5.968519,38.811881,N/A
502,ZBRA,553.240,N/A,34.79,69.306931,10.87,82.970297,5.6500,74.455446,28.032851,81.980198,11.710455,76.435644,N/A
503,ZION,70.860,N/A,6.78,9.50495,1.49,16.435644,3.1000,46.732673,5.368032,7.920792,2.883805,14.257426,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,RV Score
0,A,149.140,N/A,19.28,37.425743,4.37,56.534653,3.7300,54.455446,14.521552,48.118812,6.981359,48.712871,49.049505
1,AAL,19.525,N/A,-4.03,5.148515,-1.66,5.544554,0.5121,5.148515,-7.630454,2.376238,1.476193,6.336634,4.910891
2,AAP,246.330,N/A,24.92,52.079208,4.76,59.80198,1.4100,19.60396,14.000232,46.336634,3.122170,16.435644,38.851485
3,AAPL,178.740,N/A,15.83,28.910891,22.38,93.861386,3.9200,57.425743,12.533348,39.80198,9.767116,68.118812,57.623762
4,ABBV,135.670,N/A,33.42,65.940594,18.03,92.079208,4.4200,62.970297,11.718143,36.039604,8.119916,57.227723,62.851485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.350,N/A,27.12,55.643564,-5.14,5.148515,6.5300,78.613861,23.160349,73.465347,10.702701,72.277228,57.029703
501,ZBH,131.930,N/A,33.23,65.544554,2.20,31.089109,3.5000,51.485149,19.277595,65.940594,5.968519,38.811881,50.574257
502,ZBRA,553.240,N/A,34.79,69.306931,10.87,82.970297,5.6500,74.455446,28.032851,81.980198,11.710455,76.435644,77.029703
503,ZION,70.860,N/A,6.78,9.50495,1.49,16.435644,3.1000,46.732673,5.368032,7.920792,2.883805,14.257426,18.970297


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_df.sort_values('RV Score', ascending=True, inplace=True)
rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace = True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,RV Score
0,UNM,29.090,N/A,7.38,10.693069,0.5083,5.742574,0.4289,3.564356,3.350813,3.366337,0.415461,0.792079,4.831683
1,AAL,19.525,N/A,-4.03,5.148515,-1.6600,5.544554,0.5121,5.148515,-7.630454,2.376238,1.476193,6.336634,4.910891
2,HPQ,39.000,N/A,3.79,7.128713,-13.0600,3.762376,0.3380,1.188119,4.112379,4.950495,1.797025,8.514851,5.108911
3,PRU,118.430,N/A,6.47,8.910891,0.7077,6.534653,0.6194,5.742574,4.670552,5.940594,0.614749,1.188119,5.663366
4,BEN,36.283,N/A,4.96,7.722772,0.7949,7.524752,1.0300,12.178218,3.132181,3.168317,0.848229,2.376238,6.594059
5,KSS,48.710,N/A,7.61,11.089109,1.4200,14.950495,0.3726,2.574257,3.804095,4.158416,1.150005,4.158416,7.386139
6,BA,223.150,N/A,-15.47,4.158416,-8.7000,4.554455,1.9800,30.990099,-34.397430,0.594059,-1415.238523,0.19802,8.09901
7,DHI,99.890,N/A,4.22,7.524752,1.2500,12.079208,0.6818,6.336634,3.589614,3.762376,2.406524,11.485149,8.237624
8,AIG,63.840,N/A,9.61,14.059406,0.7988,7.722772,1.0300,12.178218,4.770629,6.336634,1.059136,3.366337,8.732673
9,HPE,17.540,N/A,3.46,6.534653,0.5609,6.138614,0.4060,3.168317,7.873792,16.831683,2.148549,11.089109,8.752475


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the value of your portfolio:25000


In [21]:
position_size = float(portfolio_size)/len(rv_df.index)
for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = position_size/rv_df.loc[row, 'Price']
    
rv_df

C:\Users\Local User\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,RV Score
0,UNM,29.090,17.188037,7.38,10.693069,0.5083,5.742574,0.4289,3.564356,3.350813,3.366337,0.415461,0.792079,4.831683
1,AAL,19.525,25.608195,-4.03,5.148515,-1.6600,5.544554,0.5121,5.148515,-7.630454,2.376238,1.476193,6.336634,4.910891
2,HPQ,39.000,12.820513,3.79,7.128713,-13.0600,3.762376,0.3380,1.188119,4.112379,4.950495,1.797025,8.514851,5.108911
3,PRU,118.430,4.221903,6.47,8.910891,0.7077,6.534653,0.6194,5.742574,4.670552,5.940594,0.614749,1.188119,5.663366
4,BEN,36.283,13.780558,4.96,7.722772,0.7949,7.524752,1.0300,12.178218,3.132181,3.168317,0.848229,2.376238,6.594059
5,KSS,48.710,10.264833,7.61,11.089109,1.4200,14.950495,0.3726,2.574257,3.804095,4.158416,1.150005,4.158416,7.386139
6,BA,223.150,2.240645,-15.47,4.158416,-8.7000,4.554455,1.9800,30.990099,-34.397430,0.594059,-1415.238523,0.19802,8.09901
7,DHI,99.890,5.005506,4.22,7.524752,1.2500,12.079208,0.6818,6.336634,3.589614,3.762376,2.406524,11.485149,8.237624
8,AIG,63.840,7.83208,9.61,14.059406,0.7988,7.722772,1.0300,12.178218,4.770629,6.336634,1.059136,3.366337,8.732673
9,HPE,17.540,28.506271,3.46,6.534653,0.5609,6.138614,0.4060,3.168317,7.873792,16.831683,2.148549,11.089109,8.752475


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [23]:
writer.save()